In [1]:
import os
import zipfile

os.system('pip install gdown')
import gdown

FILE_ID = '1LI195R4rbW7gJ_fzYtmkKu4kHE4RXtXT' 

OUTPUT_ZIP = '/kaggle/working/dataset.zip'
EXTRACT_PATH = '/kaggle/working/dataset'

if not os.path.exists(OUTPUT_ZIP) and not os.path.exists(EXTRACT_PATH):
    print("⬇️ Google Drive'dan dosya indiriliyor...")
    url = f'https://drive.google.com/uc?id={FILE_ID}'
    gdown.download(url, OUTPUT_ZIP, quiet=False)
    print("✅ İndirme tamamlandı.")

if not os.path.exists(EXTRACT_PATH):
    print("📂 Zip dosyası açılıyor...")
    with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    print("✅ Hazır! Dosyalar '/kaggle/working/dataset' klasörüne çıkarıldı.")
else:
    print("✅ Dosyalar zaten mevcut, işlem yapılmadı.")

⬇️ Google Drive'dan dosya indiriliyor...


Downloading...
From (original): https://drive.google.com/uc?id=1LI195R4rbW7gJ_fzYtmkKu4kHE4RXtXT
From (redirected): https://drive.google.com/uc?id=1LI195R4rbW7gJ_fzYtmkKu4kHE4RXtXT&confirm=t&uuid=1a8f44c1-25db-4751-995b-bc508efeaefa
To: /kaggle/working/dataset.zip
100%|██████████| 2.77G/2.77G [00:21<00:00, 128MB/s]


✅ İndirme tamamlandı.
📂 Zip dosyası açılıyor...
✅ Hazır! Dosyalar '/kaggle/working/dataset' klasörüne çıkarıldı.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import gc


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Eğitim şu cihazda yapılacak: {device}")

DATA_DIR = EXTRACT_PATH
CLASSES = sorted(os.listdir(os.path.join(DATA_DIR, 'train')))
print(f"Sınıflar: {CLASSES}")

Eğitim şu cihazda yapılacak: cuda:0
Sınıflar: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [3]:
def get_model(model_name, num_classes=7, pretrained=True):
    weights = 'DEFAULT' if pretrained else None
    
    if model_name == 'resnet50':
        model = models.resnet50(weights=weights)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        
    elif model_name == 'resnet101':
        model = models.resnet101(weights=weights)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        
    elif model_name == 'efficientnet':
        model = models.efficientnet_b0(weights=weights)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
        
    elif model_name == 'vit':
        model = models.vit_b_16(weights=weights)
        num_ftrs = model.heads.head.in_features
        model.heads.head = nn.Linear(num_ftrs, num_classes)
        
    else:
        raise ValueError(f"Model {model_name} tanımlı değil.")
        
    return model

In [4]:
#DATA LOAD

## onceki transform lar data ablation ablation oncesi
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(15),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(15),
        
        #Ablation study icin ColorJitter kullanilmistir.
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        
        # RandAugment (Genelleme yeteneği için güçlü bir teknik)
        # num_ops: uygulanacak işlem sayısı, magnitude: işlemin şiddeti
        transforms.RandAugment(num_ops=2, magnitude=9),
        
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        
        # Ablation study, Cutout / RandomErasing (Resim üzerinde rastgele kareler siler, modelin bütünü öğrenmesini sağlar)
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.33), ratio=(0.3, 3.3))
    ]),
    'val': transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


### FOCAL LOSS

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ABLASYON TESTLERI FOCAL LOSS VIT, RESNET101, EFFICIENTB0.

In [6]:
#CONFIG

K_FOLDS = 3
NUM_EPOCHS = 10 
BATCH_SIZE = 32

os.makedirs("output/figures", exist_ok=True)
os.makedirs("output/reports", exist_ok=True)
os.makedirs("output/models", exist_ok=True)

train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

full_train_dataset_aug = datasets.ImageFolder(train_dir, data_transforms['train'])

full_train_dataset_clean = datasets.ImageFolder(train_dir, data_transforms['val'])

if os.path.exists(test_dir):
    test_dataset = datasets.ImageFolder(test_dir, data_transforms['val'])
else:
    print("⚠️ Test klasörü yok, test için validation seti kullanılacak.")
    test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'val'), data_transforms['val'])

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("⚖️ Sınıf ağırlıkları hesaplanıyor...")
targets = full_train_dataset_aug.targets
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(targets), y=targets
)
class_weights = torch.tensor(class_weights_array, dtype=torch.float).to(device)
print(f"✅ Sınıf Ağırlıkları: {class_weights.cpu().numpy()}")

⚖️ Sınıf ağırlıkları hesaplanıyor...
✅ Sınıf Ağırlıkları: [ 4.368593   2.7848454  1.302129  12.440994   1.2860353  0.2133802
 10.0401   ]


In [7]:
def train_one_fold(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss, running_corrects, total = 0.0, 0, 0

        pbar = tqdm(train_loader, desc=f"    Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
            
            pbar.set_postfix({'loss': loss.item()})
        
        scheduler.step()
        epoch_train_loss = running_loss / total
        epoch_train_acc = running_corrects.double() / total
        
        model.eval()
        running_loss, running_corrects, total = 0.0, 0, 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"    Epoch {epoch+1}/{num_epochs} [Val]  ", leave=False):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total += labels.size(0)
        
        epoch_val_loss = running_loss / total
        epoch_val_acc = running_corrects.double() / total
        
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc.item())
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc.item())
        
        print(f"    Epoch {epoch+1}/{num_epochs} -> Train Acc: {epoch_train_acc:.4f} | Val Acc: {epoch_val_acc:.4f}")

        if epoch_val_acc > best_acc:
            best_acc = epoch_val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
    return best_model_wts, best_acc, history

In [8]:
experiments = [
    {'name': 'resnet50',     'optimizer': 'adam', 'lr': 1e-4}, 
    {'name': 'resnet101',    'optimizer': 'adam', 'lr': 1e-4},
    {'name': 'efficientnet', 'optimizer': 'adam', 'lr': 1e-4},
    {'name': 'vit',          'optimizer': 'sgd',  'lr': 1e-3}
]

kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

print(f"\n🚀 {K_FOLDS}-FOLD CROSS VALIDATION SÜRECİ BAŞLIYOR...")
print("="*60)

for exp in experiments:
    model_name = exp['name']
    print(f"\n▶️  MODEL: {model_name.upper()}")

    torch.cuda.empty_cache()
    gc.collect()
    
    fold_accuracies = []
    best_global_acc = 0.0
    best_global_wts = None
    
    for fold, (train_ids, val_ids) in enumerate(kfold.split(full_train_dataset_aug)):
        print(f"   ↳ Fold {fold+1}/{K_FOLDS}...")
        
        train_sub = Subset(full_train_dataset_aug, train_ids)
        val_sub = Subset(full_train_dataset_clean, val_ids)
        
        train_loader_fold = DataLoader(train_sub, batch_size=BATCH_SIZE, shuffle=True)
        val_loader_fold = DataLoader(val_sub, batch_size=BATCH_SIZE, shuffle=False)
        
        model = get_model(model_name, num_classes=7, pretrained=True).to(device)
        criterion = FocalLoss(alpha=class_weights, gamma=2.0)
        
        if exp['optimizer'] == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=exp['lr'])
        else:
            optimizer = optim.SGD(model.parameters(), lr=exp['lr'], momentum=0.9)
        
        ##ablasyon cosine annealing
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        
        wts, acc, hist = train_one_fold(
            model, criterion, optimizer, scheduler, 
            train_loader_fold, val_loader_fold, NUM_EPOCHS
        )
        
        fold_accuracies.append(acc.item())
        print(f"      ✅ Fold {fold+1} Best Val Acc: {acc:.4f}")
        
        if acc > best_global_acc:
            best_global_acc = acc
            best_global_wts = wts

        del model, optimizer, scheduler
        torch.cuda.empty_cache()
        gc.collect()

    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    print(f"\n📊 {model_name.upper()} CV Sonucu: {mean_acc:.4f} ± {std_acc:.4f}")
    
    print(f"🧪 Hold-out Test Seti Değerlendiriliyor (En iyi CV modeli ile)...")
    
    final_model = get_model(model_name, num_classes=7, pretrained=False).to(device)
    final_model.load_state_dict(best_global_wts)
    final_model.eval()
    
    torch.save(final_model.state_dict(), f"output/models/final_{model_name}_best.pth")
    
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = final_model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    report = classification_report(all_labels, all_preds, target_names=CLASSES)
    cm = confusion_matrix(all_labels, all_preds)
    
    with open(f"output/reports/final_{model_name}_cv_report.txt", "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"CV Accuracy: {mean_acc:.4f} +/- {std_acc:.4f}\n")
        f.write("="*60 + "\n")
        f.write("TEST SET RESULTS:\n")
        f.write(report)
        
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
    plt.title(f'{model_name} Final Test CM (CV Avg: {mean_acc:.2f})')
    plt.savefig(f"output/figures/final_{model_name}_cv_cm.png")
    plt.close()
    
    print(f"✅ {model_name} işlemleri tamamlandı. Rapor kaydedildi.\n")

    del final_model
    torch.cuda.empty_cache()
    gc.collect()


🚀 3-FOLD CROSS VALIDATION SÜRECİ BAŞLIYOR...

▶️  MODEL: RESNET50
   ↳ Fold 1/3...
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 193MB/s]


    Epoch 1/10 [Train]:   0%|          | 0/167 [00:00<?, ?it/s]

    Epoch 1/10 [Val]  :   0%|          | 0/84 [00:00<?, ?it/s]

    Epoch 1/10 -> Train Acc: 0.1436 | Val Acc: 0.2115


...

    Epoch 9/10 [Val]  :   0%|          | 0/84 [00:00<?, ?it/s]

    Epoch 9/10 -> Train Acc: 0.4204 | Val Acc: 0.5431


    Epoch 10/10 [Train]:   0%|          | 0/167 [00:00<?, ?it/s]

    Epoch 10/10 [Val]  :   0%|          | 0/84 [00:00<?, ?it/s]

    Epoch 10/10 -> Train Acc: 0.4386 | Val Acc: 0.5543
      ✅ Fold 3 Best Val Acc: 0.5566

📊 VIT CV Sonucu: 0.5331 ± 0.0344
🧪 Hold-out Test Seti Değerlendiriliyor (En iyi CV modeli ile)...
✅ vit işlemleri tamamlandı. Rapor kaydedildi.



Model: efficientnet
CV Accuracy: 0.4668 +/- 0.0247
============================================================
TEST SET RESULTS:
              
              precision    recall  f1-score   support

       akiec       0.28      0.72      0.40        32
         bcc       0.50      0.79      0.61        52
         bkl       0.45      0.62      0.52       110
          df       0.15      0.64      0.25        11
         mel       0.23      0.65      0.34       112
          nv       0.99      0.41      0.58       671
        vasc       0.27      1.00      0.42        14

    accuracy                           0.50      1002
    macro avg      0.41      0.69      0.45      1002
    weighted avg   0.78      0.50      0.54      1002

Model: resnet101
CV Accuracy: 0.5779 +/- 0.0084
============================================================
TEST SET RESULTS:
              
              precision    recall  f1-score   support

       akiec       0.36      0.84      0.50        32
         bcc       0.67      0.90      0.77        52
         bkl       0.51      0.66      0.57       110
          df       0.33      0.82      0.47        11
         mel       0.32      0.80      0.46       112
          nv       0.99      0.55      0.70       671
        vasc       0.37      0.93      0.53        14

    accuracy                           0.62      1002
    macro avg      0.51      0.79      0.57      1002
    weighted avg   0.81      0.62      0.65      1002

Model: resnet50
CV Accuracy: 0.5401 +/- 0.0106
============================================================
TEST SET RESULTS:
          
              precision    recall  f1-score   support

       akiec       0.31      0.72      0.43        32
         bcc       0.58      0.77      0.66        52
         bkl       0.49      0.64      0.56       110
          df       0.25      1.00      0.40        11
         mel       0.25      0.64      0.36       112
          nv       0.98      0.53      0.69       671
        vasc       0.43      0.93      0.59        14

    accuracy                           0.58      1002
    macro avg      0.47      0.75      0.53      1002
    weighted avg   0.79      0.58      0.62      1002

Model: vit
CV Accuracy: 0.5331 +/- 0.0344
============================================================
TEST SET RESULTS:

              precision    recall  f1-score   support

       akiec       0.35      0.19      0.24        32
         bcc       0.67      0.56      0.61        52
         bkl       0.45      0.61      0.52       110
          df       0.14      0.82      0.24        11
         mel       0.27      0.79      0.40       112
          nv       0.98      0.53      0.69       671
        vasc       0.39      0.93      0.55        14

    accuracy                           0.57      1002
    macro avg      0.47      0.63      0.47      1002
    weighted avg   0.79      0.57      0.61      1002